# Abstract

Try out various perfomance checks and issues

# Environment

# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [1]:
%load_ext line_profiler

In [2]:
import cProfile
import pdb

In [3]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level2_only,
    registry_level3_only
)

In [4]:
from jwst.associations import (
    AssociationRegistry,
    AssociationPool,
    generate
)

In [5]:
from jwst.associations.main import (
    constrain_on_candidates
)

# Library

# Main

## Timings

### Dead simple: 1 row, 1 rule

In [6]:
pool_one = combine_pools('data/pool_012_image_miri_single_exp.csv')

/Users/eisenham/anaconda3/envs/asnopt_py3/lib/python3.5/site-packages/astropy/table/table.py:1301: StringTruncateWarning: truncated right side string(s) longer than 14 character(s) during assignment
  col[item] = val


In [7]:
rules_lv3_imageonly = registry_level3_only()
names = [name for name in rules_lv3_imageonly]
for name in names:
    if name != 'Asn_Image':
        del rules_lv3_imageonly[name]

In [8]:
run_1x1 = %timeit -o generate(pool_one, rules_lv3_imageonly)

4.95 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### One rule, full pool

In [9]:
pool_full = AssociationPool.read('data/mega_pool.csv')

In [10]:
run_fullx1 = %timeit -o generate(pool_full, rules_lv3_imageonly)

52.8 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print('Estimated run time = {}'.format(
    len(pool_full) * run_1x1.average)
)

Estimated run time = 0.5642257671103912


In [12]:
print('member/sec = {}'.format(
    len(pool_full) / run_fullx1.average
))

member/sec = 2160.219724969652


### 1 row, full rules

In [13]:
rules_full = AssociationRegistry()

In [14]:
run_1xfull = %timeit -o generate(pool_one, rules_full)

7.7 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
print('Processing time for member per rule = {}'.format(
     run_1xfull.average / len(rules_full)
))

Processing time for member per rule = 0.0006418683954827776


In [16]:
print('Rules/sec = {}'.format(
    len(rules_full) / run_1xfull.average
))

Rules/sec = 1557.951765560689


### Full rules, Full pool

In [17]:
print('Predicted time = {}'.format(
    len(pool_full) * len(rules_full) * run_1x1.average
))

Predicted time = 6.770709205324694


In [18]:
run_fullxfull = %timeit -o generate(pool_full, rules_full)

1.49 s ± 87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
print('member/rule per sec = {}'.format(
    len(pool_full) * len(rules_full) / run_fullxfull.average
))

member/rule per sec = 919.8918108919365


In [20]:
asns, orphaned = generate(pool_full, rules_full)

In [21]:
len(asns)

13

### Add constraint

In [22]:
c_o001 = {'asn_candidate': constrain_on_candidates(['o001'])}

#### 1 row, 1 rule

In [23]:
rules_lv3_imageonly_asnid = registry_level3_only(global_constraints=c_o001)
names = [name for name in rules_lv3_imageonly_asnid]
for name in names:
    if name != 'Asn_Image':
        del rules_lv3_imageonly_asnid[name]

In [24]:
run_1x1_asnid = %timeit -o generate(pool_one, rules_lv3_imageonly_asnid)

4.88 ms ± 355 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### One rule, full pool

In [25]:
run_fullx1_asnid = %timeit -o generate(pool_full, rules_lv3_imageonly_asnid)

812 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
print('member rule time = {}'.format(
    run_fullx1_asnid.average / len(pool_full)
))

member rule time = 0.007120552468648995


In [27]:
print ('members/sec = {}'.format(
    len(pool_full) / run_fullx1_asnid.average
))

members/sec = 140.43854102654103


#### 1 row, full rules

In [28]:
rules_full_asnid = AssociationRegistry(global_constraints=c_o001)

In [29]:
run_1xfull_asnid = %timeit -o generate(pool_one, rules_full_asnid)

7.86 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
print('member per rule = {}'.format(
    run_1xfull_asnid.average / len(rules_full_asnid)
))

member per rule = 0.000654883387265034


In [45]:
print('Rule/sec = {}'.format(
    len(rules_full_asnid) / run_1xfull_asnid.average
))

Rule/sec = 1526.9894143692727


#### Full rules, Full pool

In [32]:
run_fullxfull_asnid = %timeit -o generate(pool_full, rules_full_asnid)

1.7 s ± 38.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
print('Full member rule processing = {}'.format(
    run_fullxfull_asnid.average / len(pool_full) / len(rules_full_asnid)
))

Full member rule processing = 0.0012414273922319475


In [34]:
print('member/rule per sec = {}'.format(
    len(pool_full) * len(rules_full_asnid) / run_fullxfull_asnid.average
))

member/rule per sec = 805.5243554776989


In [35]:
asns, orphaned = generate(pool_full, rules_full)

## The most basic profile

In [36]:
pool = combine_pools('data/pool_012_image_miri_single_exp.csv')

/Users/eisenham/anaconda3/envs/asnopt_py3/lib/python3.5/site-packages/astropy/table/table.py:1301: StringTruncateWarning: truncated right side string(s) longer than 14 character(s) during assignment
  col[item] = val


In [37]:
rules = registry_level3_only()

In [38]:
asns, orphaned = generate(pool, rules)

In [39]:
cProfile.run('asns, orphaned = generate(pool, rules)')

         16183 function calls (14872 primitive calls) in 0.009 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.009    0.009 <string>:1(<module>)
       24    0.000    0.000    0.000    0.000 <string>:12(__new__)
       25    0.000    0.000    0.000    0.000 __init__.py:1258(debug)
       25    0.000    0.000    0.000    0.000 __init__.py:1502(getEffectiveLevel)
       25    0.000    0.000    0.000    0.000 __init__.py:1516(isEnabledFor)
        6    0.000    0.000    0.000    0.000 __init__.py:1841(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:243(load)
        1    0.000    0.000    0.000    0.000 __init__.py:271(loads)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:594(get)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:613(items)
        2    0.000

In [40]:
asns

[{
     "asn_rule": "Asn_Image",
     "code_version": "0.7.1.beta2",
     "degraded_status": "No known degraded exposures in association.",
     "products": [
         {
             "members": [
                 {
                     "exposerr": null,
                     "expname": "jw_00001_cal.fits",
                     "asn_candidate": "[('o001', 'OBSERVATION')]",
                     "exptype": "SCIENCE"
                 }
             ],
             "name": "jw99009-a3001_t001_miri_f560w"
         }
     ],
     "target": "1",
     "asn_type": "image3",
     "version_id": null,
     "program": "99009",
     "asn_id": "a3001",
     "asn_pool": "pool_012_image_miri_single_exp",
     "constraints": "Constraints:\n    exp_type: MIR_IMAGE\n    opt_elem2: None\n    wfsvisit: Is Invalid\n    program: 99009\n    target: 1\n    opt_elem: F560W\n    instrument: MIRI"
 }]

## Full script profile

In [41]:
%prun generate(pool_full, rules_full)

## Full line profile

In [42]:
from jwst.associations import Association

In [43]:
%lprun -f Association.match_member generate(pool_full, rules_full)